In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import gurobipy as gb
import os
from pickle import dump, load
from time import process_time
from copy import deepcopy
from source import*

path = os.getcwd()

In [ ]:
file = open(path+"/Results/Configurations/Open Stations/open_stations_18","rb"); S = load(file); file.close()
K_s = {}                # {scenario:{station:[vehicles]}}n",
K = {}                  # {scenatio:[vehicles]}n",

for sc in range(25):
    file = open(path+f"/Data/K/K_sc{sc}", "rb")
    K[sc] = load(file); file.close()

    file = open(path+f"/Data/K_s10/Ks_sc{sc}", "rb")
    K_s[sc] = load(file); file.close()

original_stress = {}
for s in S:
    original_stress[s] = np.mean(np.array([len(K_s[sc][s]) for sc in range(25)]))

max_s = max(original_stress, key=original_stress.get)
max_s, original_stress[max_s]

In [ ]:
def generate_routes_given_number_chargers(s,S,y,K,K_s_prime,exhaust=False):

    total_serviced_costumers:dict[list[list]] = dict()
    
    for sc in range(25):
        routes = list()
        K, _, S_k, a, t = load_pickle(path+"/Data/",sc)
        K_s = K_s_prime[sc]

        S_k = {k:[ss for ss in S_k[k] if ss in S] for k in K_s[s]}
        a.update({("s",s):0}); a.update({("e",s):40})
        t.update({("s",s):0}); t.update({("e",s):0})
    
        mp = gb.Model("Restricted Master Problem")
    
        dummy_0 = {k:mp.addVar(vtype=gb.GRB.CONTINUOUS, obj=1, name=f"st0_{k}") for k in K_s[s]}
        aux = mp.addVar(vtype=gb.GRB.CONTINUOUS, obj=2, name=f"aux_{s}")
    
        vehic_assign = {}
        for k in K_s[s]:
            vehic_assign[k] = mp.addLConstr(dummy_0[k] == 1, name=f"V{k}_assignment")
    
        st_conv = mp.addConstr(aux <= y, f"S{s}_convexity")
    
        mp.setParam("OutputFlag",0)
    
        time0 = process_time(); i = 0
        lbd = []; lastMP = 1e3
    
        while True:
            
            mp.update()
            mp.optimize()
    
            pi = {k:vehic_assign[k].getAttr("Pi") for k in K_s[s]}
            sigma = st_conv.getAttr("Pi")
            thisMP = mp.getObjective().getValue()
    
            if (lastMP-thisMP < 0.1 and thisMP > 5) or (lastMP-thisMP < 0.01 and thisMP<=5):
                #if y == 8: print(f"ttOut of Labeling scenario {sc}")
                mpsol = thisMP; break
    
            #print(f"tIteration {i}:ttMP obj: {round(mp.getObjective().getValue(),2)}ttime: {round(process_time()-time0,2)}s")
            i += 1
    
            V,A,rc = get_graph_chargers(s,K_s[s],a,pi,sigma)
            ext = vertices_extensions(V,A)
            
            #if y == 8:
            #    print(f"ttIteration {i}. Labeling for scenario {sc}")
    
            routes_DFS = {"s":([0],0)}; covered_nodes = {v:0 for v in V}; best_len = {v:0 for v in V}; best_t = {v:0 for v in V}; best_h = {v:30 for v in V}; num_improv = {v:0 for v in V}; tried = {v:0 for v in V}
            marked = {}; max_successors = {v:100 for v in V}
            mod_label_DFS(v="s",rP=0,tP=0,qP=0,hP=0,P=[],cK=covered_nodes,L=routes_DFS,s=s,r=rc,t=t,a=a,ext=ext,best_len=best_len,best_t=best_t,best_h=best_h,tried=tried,marked=marked,num_improv=num_improv,max_successors=max_successors)
            del routes_DFS["s"]
            
            opt = 0
            for l in routes_DFS:
                if routes_DFS[l][1] < -0.001:
                    col = {k:1 if k in routes_DFS[l][0] else 0 for k in K_s[s]}
                    new_Col = gb.Column(col.values(),vehic_assign.values())
                    lbd.append(mp.addVar(vtype=gb.GRB.CONTINUOUS,obj=0,column=new_Col))
    
                    mp.chgCoeff(st_conv,lbd[-1],1)
                    opt +=1
            if i == 1:
                original_routes = routes_DFS
    
            if opt == 0 : mpsol = thisMP; break
            lastMP = thisMP
        
        for v in mp.getVars():
            v.vtype = gb.GRB.BINARY
        #if mpsol < len(K_s[s]): mp.setParam("MIPGap",0.01)
        mp.setParam("MIPFocus",2)
        mp.update(); mp.optimize()
    
        print(f"ttSolved IMP scenario {sc}, with {mp.getObjective().getValue()} unnasigned vehicles")
        # print(f"IMP solution: {mp.getObjective().getValue()} assigned vehicles with {y} chargers")
        for l in lbd:
            if l.X > 0.5:
                col = [k for k in K_s[s] if mp.getCoeff(vehic_assign[k],l) == 1]
                routes.append(col)
    
        total_serviced_costumers[sc]:list = list()
    
        for route in routes:
            total_serviced_costumers[sc].extend(route)
        
        if len(total_serviced_costumers[sc]) < len(K_s_prime[sc][s]) and y < 8:
            print(f't{y} number of chargers is infeasiblen')
            return None
    
    return total_serviced_costumers

In [ ]:
ordered_stations:list = sorted(original_stress, key = original_stress.get, reverse = True)     # Get list of stations ordered by stress
number_of_chargers:dict = dict()
stress = deepcopy(original_stress)

K_prime = deepcopy(K)
K_s_prime = deepcopy(K_s) 
# S_k_prime = deepcopy(S_k)

cont:int = 0

while True:
    cont += 1 
    s = ordered_stations[0]
    print(f'--------- Station {s} - {stress[s]} - {original_stress[s]} ---------')
    y:int = 1       # Start giving one charger to the station

    # Iterative process to find number of chargers 
    while True:
        print(f"\tTrying for {y} chargers for station {s}")
        total_serviced_costumers:dict[list[list]] or None = generate_routes_given_number_chargers(s, S, y, K_prime, K_s_prime, False)     # Get the routes for the stations
        
        if total_serviced_costumers == None:  # Station doesn't service all possible costumers in at least one scenario
            y += 1
        else:               # Number of chargers is fixed
            break
    
    # Set number of chargers and remove already serviced vehicles
    number_of_chargers[s] = y
    print(f'\n\tStation {s} set to {number_of_chargers[s]} chargers\nn----------------------------')

    
    for u in range(25):
        for vehicle in total_serviced_costumers[u]:
            K_prime[u].remove(vehicle)
        
            for ss in S:
                if vehicle in K_s_prime[u][ss]:
                    K_s_prime[u][ss].remove(vehicle)

    S.remove(s)
    stress:dict = {}
    for s in S:
        stress[s] = np.mean(np.array([len(K_s_prime[sc][s]) for sc in range(25)]))    
    ordered_stations:list = sorted(stress, key = stress.get, reverse = True)     # Get list of stations ordered by stress

    if cont == 600 or sum(len(K_s_prime[sc][s]) for sc in range(25) for s in S)==0:
        break